## 数组不相邻元素最大和

$$
\begin{array}{l}
0 & 1 & 2 & 3 & 4 & 5 & 6 \\
1 & 2 & 4 & 1 & 7 & 8 & 3
\end{array}
$$
递归表达式：

$$
OPT(i) = max
\begin{cases}
OPT(i-2)+arr\left[i\right]\\
OPT(i-1)
\end{cases}
$$

递归出口：

$$
\begin{align*}
OPT(0) &= arr\left[0\right]\\\\
OPT(1) &= max
\begin{cases}
arr\left[0\right]\\
arr\left[1\right]
\end{cases}
\end{align*}
$$

In [ ]:
arr = [1, 2, 4, 1, 7, 8, 3]  # array

# recursive
# O(N^2)
def rec_opt(arr, i):
    if 0 == i:
        return arr[0]
    elif 1 == i:
        return max(arr[0], arr[1])
    else:
        A = rec_opt(arr, i - 2) + arr[i]
        B = rec_opt(arr, i - 1)
        return max(A, B)

print(rec_opt(arr, len(arr) - 1))

# recursive with memory
import numpy as np
memo = np.zeros(len(arr)) - 1
memo[0] = arr[0]

def rec_opt_mem(arr, i):
    if memo[i] != -1:
        return memo[i]

    if 0 == i:
        return arr[0]
    elif 1 == i:
        memo[i] = max(arr[0], arr[1])
        return memo[i]
    else:
        A = rec_opt_mem(arr, i - 2) + arr[i]
        B = rec_opt_mem(arr, i - 1)
        memo[i] = max(A, B)
        return memo[i]

print(rec_opt_mem(arr, len(arr) - 1))

# iterative, dynamic programming
# O(N)
def dp_opt(arr):
    opt = np.zeros(len(arr))
    opt[0] = arr[0]
    opt[1] = max(arr[0], arr[1])
    for i in range(2, len(arr)):
        A = opt[i - 2] + arr[i]
        B = opt[i - 1]
        opt[i] = max(A, B)
    return opt[-1]

print(dp_opt(arr))

## N-Sum问题

已知数组 arr = \[3, 34, 4, 12, 5, 2\]

能否找出其中和为 *S = 9* 的所有元素？
能就返回 *true*，否则返回 *false*

递归表达式：

Subset(i, s) = Subset(i - 1, s - arr\[i\]) **or** Subset(i - 1, s)

递归出口：

if s == 0: return True

if i == 0: return arr\[0\] == s

if arr\[i\] > s: return Subset(i - 1, s)

In [ ]:
arr = [3, 34, 4, 12, 5, 2]

def rec_subset(arr, i, s):
    if 0 == s:
        return True
    elif 0 == i:
        return arr[0] == s
    elif arr[i] > s:
        return rec_subset(arr, i - 1, s)
    else:
        return rec_subset(arr, i - 1, s - arr[i]) or rec_subset(arr, i - 1, s)

print(rec_subset(arr, len(arr)-1, 9))
print(rec_subset(arr, len(arr)-1, 10))
print(rec_subset(arr, len(arr)-1, 11))
print(rec_subset(arr, len(arr)-1, 12))
print(rec_subset(arr, len(arr)-1, 13))

构造矩阵法：
$$
\begin{array}{r}
arr & i & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
3   & 0 & T & F & F & T & F & F & F & F & F & F \\
34  & 1 & T & \cdots \\
4   & 2 & T &   & \cdots \\
12  & 3 & T &   &   & \cdots \\
5   & 4 & T &   &   &   & \cdots \\
2   & 5 & T &   &   &   &   & \cdots
\end{array}
$$

In [ ]:
import numpy as np
def dp_subset(arr, S):
    subset = np.zeros((len(arr), S + 1), dtype=bool)
    subset[0, :] = False
    subset[:, 0] = True
    subset[0, arr[0]] = True
    for i in range(1, len(arr)):
        for s in range(1, S + 1):
            if arr[i] > s:
                subset[i, s] = subset[i - 1, s]
            else:
                subset[i, s] = subset[i - 1, s - arr[i]] or subset[i - 1, s]
    return subset[-1, -1]

print(dp_subset(arr, 9))
print(dp_subset(arr, 10))
print(dp_subset(arr, 11))
print(dp_subset(arr, 12))
print(dp_subset(arr, 13))

## 挖金矿

有一个国家发现了5座金矿，每座金矿的黄金储量不同，需要参与挖掘的工人数也不同。参与挖矿工人总数是10人。每座金矿要么全挖，要么不挖，不能派出一半人挖取一半金矿。要想得到尽可能多的黄金，应该选择挖取哪几座金矿？

$$
\begin{array}{c}
500金 & 200金 & 300金 & 350金 & 400金 \\
  5人 &   3人 &   4人 &   3人 &   5人
\end{array}
$$

**方法一：排列组合**

每一座金矿都有挖与不挖两种选择，如果有N座金矿，排列组合有2^N种选择。对所有可能性做遍历，排除那些使用工人数超过10的选择，在剩下的选择里找出获得金币数最多的选择。

时间复杂度为O(2^N)。

In [ ]:
import itertools

g = [500, 200, 300, 350, 400]   # gold
w = [5, 3, 4, 3, 5]             # worker

index = list(range(0, len(g)))
gs = 0  # 黄金总数
for r in range(1, len(index)+1):
    # 选择不同数量时的组合数
    for ind in itertools.combinations(index, r):
        ws = sum([w[i] for i in ind])    # 工人总数
        if ws > 10:
            pass
        else:
            gs = max(gs, sum([g[i] for i in ind]))   # 挖取的黄金总数
print(gs)

动态规划

递归表达式：
$$
OPT(i, N)=max
\begin{cases}
OPT(i-1,N-w\left[i\right]) + g\left[i\right],\\
OPT(i-1, N)
\end{cases},
$$
N 为总人数

递归出口：

（1）只有一座金矿，只能挖这唯一的，而且人数不能超，得到的黄金就是这个矿的

$OPT(0, N) = g\left[0\right], \quad if\quad i == 0 \quad \& \quad N \geq w\left[0\right]$

（2）如果给定的工人数量不够挖取第1座金矿，也就是 *N < w\[0\]* 的时候，收获为0

$OPT(0, N) = 0, \quad if\quad i == 0 \quad \&\quad  N < w\left[0\right]$

（3）可用工人数小于挖取该金矿需要人数

$OPT(i, N) = OPT(i - 1, N), \quad if\quad i \geq 1 \quad \&\quad  N < w\left[i\right]$

In [ ]:
g = [400, 500, 200, 300, 350]
w = [5, 5, 3, 4, 3]

def rec_opt(g, i, w, n):
    # i 是金矿数
    # n 是工人数
    if 0 == i and n < w[0]:
        return 0
    if 0 == i and n >= w[0]:
        return g[0]
    if i > 0 and n < w[i - 1]:
        return rec_opt(g, i - 1, w, n)
    
    a = rec_opt(g, i - 1, w, n)
    b = rec_opt(g, i - 1, w, n - w[i]) + g[i]
    return max(a, b)

print(rec_opt(g, len(g)-1, w, 10))

In [ ]:
import numpy as np
# 动态规划，二维数组法

g = [400, 500, 200, 300, 350]
w = [5, 5, 3, 4, 3]

def dp_subset(g, ng, w, n):
    # ng is number of gold mines
    # n is number of workers
    col = n + 1
    preResults = np.zeros(col)    # 存放上一行结果
    results = np.zeros(col)       # 存放当前行结果

    # 填充边界
    for i in range(0, col):
        if i < w[0]:
            preResults[i] = 0
        else:
            preResults[i] = g[0]
    
    # 填充其余格子，从上一行推出下一行，外层循环金矿数量，内层工人数
    for i in range(0, ng):
        for j in range(0, col):
            if j < w[i]:
                results[j] = preResults[j]
            else:
                results[j] = max(preResults[j], preResults[j - w[i]] + g[i])
        
        for j in range(0, col):
            preResults[j] = results[j]
    
    return results[-1]

dp_subset(g, len(g), w, 10)

## 跳台阶

一只青蛙一次可以跳上1级台阶，也可以跳上2级。问该青蛙跳上一个n级的台阶总共有多少种跳法（先后次序不同算不同的结果）。

设 *f(n)* 表示青蛙跳上n级台阶的跳法数。

当只有一个台阶时，即 *n = 1* 时，只有 *1* 种跳法；

当 *n = 2* 时，有 *2* 种跳法；

当 *n = 3* 时，有 *3* 种跳法；

当 *n* 很大时，青蛙在最后一步跳到第 *n* 级台阶时，有两种情况：

一种是青蛙在第 *n-1* 个台阶跳一个台阶，那么青蛙完成前面 *n-1* 个台阶，就有 *f(n-1)* 种跳法，这是一个子问题。

另一种是青蛙在第 *n-2* 个台阶跳两个台阶到第 *n* 个台阶，那么青蛙完成前面 *n-2* 个台阶，就有 *f(n-2)* 种情况，这又是另外一个子问题。

两个子问题构成了最终问题的解，所以当 *n >= 3* 时，青蛙就有 *f(n) = f(n-1) + f(n-2)* 种跳法。

上面的分析过程，其实我们用到了动态规划的方法，找到了状态转移方程，用数学方程表达如下：

$$
f(n)=\begin{cases}
1, &n=1\\
2, &n=2\\
f(n-1)+f(n-2), &n \geq 3
\end{cases}
$$

这就是斐波那契数列。

## 三角形最短路径

给定一个如下所示三角形，找出从顶到底的路径，使得所有数字加和最小。

\[

       [2],
      [3,4],
     [6,5,7],
    [4,1,8,3]
    
\]

最小加和为11（2+3+5+1=11）

递归公式：

$$
dp_{i,j} = min
\begin{cases}
dp_{i+1,j}\\
dp_{i+1,j+1}
\end{cases}
+arr_{i,j}
$$

In [ ]:
import numpy as np
import sys

triangle = [[2], [3, 4], [6, 5, 7],[4, 1, 8, 3]]

# 动态规划
def dp_minSum(triangle):
    if len(triangle) == 1:
        return triangle[0][0]
    
    dp = np.zeros(len(triangle[-1]))

    for i in range(0, len(triangle[-1])):
        dp[i] = triangle[-1][i]
    
    for i in range(len(triangle) - 2, -1, -1):
        for j in range(0, len(triangle[i])):
            dp[j] = min(dp[j], dp[j+1]) + triangle[i][j]
    
    return dp[0]

print(dp_minSum(triangle))

# 递归
def rec_minSum(triangle, curSum, minimum, index, level):
    curSum += triangle[level][index]
    if level == len(triangle) - 1:
        return min(minimum, curSum)
    
    a = rec_minSum(triangle, curSum, minimum, index, level + 1)
    b = rec_minSum(triangle, curSum, minimum, index + 1, level + 1)
    return min(a, b)

print(rec_minSum(triangle, 0, sys.maxsize, 0, 0))

# 优化参数的递归
def rec_minSum_opt(triangle, row, col):
    if row == len(triangle) - 1:
        return triangle[row][col]
    
    a = rec_minSum_opt(triangle, row + 1, col)
    b = rec_minSum_opt(triangle, row + 1, col + 1)
    return min(a, b) + triangle[row][col]

print(rec_minSum_opt(triangle, 0, 0))

# 带“记忆”的递归
memo = np.zeros((len(triangle) * (1 + len(triangle))) // 2) -1

def rec_minSum_mem(triangle, row, col):
    
    if row == len(triangle) - 1:
        return triangle[row][col]
    
    if memo[row * (row + 1) // 2 + col] != -1:
        return memo[row * (row + 1) // 2 + col]
    else:
        a = rec_minSum_mem(triangle, row + 1, col) + triangle[row][col]
        b = rec_minSum_mem(triangle, row + 1, col + 1) + triangle[row][col]
        minValue = min(a, b)
        memo[row * (row + 1) // 2 + col] = minValue
        return minValue

print(rec_minSum_mem(triangle, 0, 0))

## Minimum Path Sum

**Description：**

Given a m x n grid filled with non-negative numbers, find a path from top left to bottom right which minimizes the sum of all numbers along its path.

**Note:** You can only move either down or right at any point in time.

**Example:**

```
Input:

[
    [1,3,1],
    [1,5,1],
    [4,2,1]
]

Output: 7
```
**Explanation:** Because the path 1→3→1→1→1 minimizes the sum.

**解题思路：**

（1） 递归法

**问题分析：**

1）假如我们就在最右下角的格子(也可以想象成网格只有一个格子)，那么最短路径和就是格子中的值；

2）假如我们在最后一行的格子中，假如是grid\[grid.length - 1\]\[col\]，那么从这个点出发到最右下角的最小路径和就是它本身加上它右边的格子到最右下角的最小路径和。

3）最后一列和最后一行是同理的。

4）一个普通的位置，它到最右下角的最小路径和是多少呢，是它左边一个位置和它下面一个位置的最小路径和中最小的那个加上它自己格子的值。

In [ ]:
# 递归代码实现
arr = [[1, 3, 1], [1, 5, 1], [4, 2, 1]]

def rec_pathSum(grid, row, col):
    if row == len(grid) - 1 and col == len(grid[0]) - 1:
        return grid[row][col]
    
    if row == len(grid) - 1:
        return grid[row][col] + rec_pathSum(grid, row, col + 1)
    
    if col == len(grid[0]) - 1:
        return grid[row][col] + rec_pathSum(grid, row + 1, col)
    
    return grid[row][col] + min(rec_pathSum(grid, row + 1, col),
                                rec_pathSum(grid, row, col + 1))

print(rec_pathSum(arr, 0, 0))

（2）记忆化搜索

既然上面的过程有很多重复计算的子问题，那我们可以在递归的过程中记录子问题的解，如果之前已经求解过，使用一个二位数组记录一下，那么我们下次再需要这个子问题的解的时候不需要递归，直接拿出来用即可。

In [ ]:
import numpy as np

arr = [[1, 3, 1], [1, 5, 1], [4, 2, 1]]
memo = np.zeros((len(arr), len(arr[0]))) - 1

def rec_pathSum_mem(grid, row, col):
    if row == len(grid) - 1 and col == len(grid[0]) - 1:
        return grid[row][col]
    
    if memo[row][col] != -1:
        return memo[row][col]
    
    if row == len(grid) - 1:
        minValue = grid[row][col] + rec_pathSum_mem(grid, row, col + 1)
    elif col == len(grid[0]) - 1:
        minValue = grid[row][col] + rec_pathSum_mem(grid, row + 1, col)
    else:
        minValue = grid[row][col] + min(rec_pathSum_mem(grid, row + 1, col),
                                        rec_pathSum_mem(grid, row, col + 1))
    
    memo[row][col] = minValue
    return minValue    

print(rec_pathSum_mem(arr, 0, 0))

（3）动态规划法

动态规划的过程可以看做是递归的逆过程，既然递归是从上往下求解，每个大的问题都要先去求解子问题。所以，动态规划是先求解小的子问题，依次往上，所以大问题需要的子问题已经提前求好了。 

对于这个题目：

先生成一张二维dp表，然后按照递归相反的方向求解；
先把dp表的最右下角，最后一行，和最后一列的位置填好；
然后一个其它的位置依赖它下面的位置和右边的位置，所以我们依次从下到上，做右往左，填整张dp表，最后dp[0][0]就是答案。 

r = grid.length - 1

c = grid\[0\].length - 1

a. 最右下角的格子：dp\[r\]\[c\] = grid\[r\]\[c\]

b. 最后一行的dp表：dp\[r\]\[j\] = grid\[r\]\[j\] + dp\[r\]\[j+1\]

c. 最后一列的dp表：dp\[i\]\[c\] = grid\[i\]\[c\] + dp\[i+1\]\[c\]

d. 普通位置：dp\[i\]\[j\] = min(dp\[i+1\]\[j\], dp\[i\]\[j+1\]) + grid\[i\]\[j\]

In [ ]:
# 动态规划

import numpy as np

arr = [[1, 3, 1], [1, 5, 1], [4, 2, 1]]

def dp_pathSum(grid):
    row = len(grid)
    col = len(grid[0])
    dp = np.zeros((row, col))
    # 最右下角格子
    dp[row - 1][col - 1] = grid[row - 1][col - 1]
    # 填充最后一行
    for i in range(col - 2, -1, -1):
        dp[row - 1][i] = dp[row - 1][i + 1] + grid[row - 1][i]
    # 填充最后一列
    for i in range(row - 2, -1, -1):
        dp[i][col - 1] = dp[i + 1][col - 1] + grid[i][col - 1]
    # 其它格子
    for i in range(row - 2, -1, -1):
        for j in range(col - 2, -1, -1):
            dp[i][j] = min(dp[i + 1][j], dp[i][j + 1]) + grid[i][j]
    
    return dp[0][0]

print(dp_pathSum(arr))

## Integer Break

**Description:**

Given a positive integer n, break it into the sum of **at least** two positive integers and maximize the product of those integers. Return the maximum product you can get.

**Example 1:**
```
Input: 2
Output: 1
Explanation: 2 = 1 + 1, 1 × 1 = 1.
```
**Example 2:**
```
Input: 10
Output: 36
Explanation: 10 = 3 + 3 + 4, 3 × 3 × 4 = 36.
```
**Note:** You may assume that n is not less than 2 and not larger than 58.

**解题思路：**

**发现重叠子问题**

In [ ]:
# 递归

import sys

def rec_intergerBreak(n):
    maxProduct = -sys.maxsize - 1
    
    if n == 2 or n == 1:
        return 1
    
    for i in range(1, n):
        maxProduct = max(maxProduct, max(i * (n - i), i * rec_intergerBreak(n - i)))
    
    return maxProduct

print(rec_intergerBreak(10))

In [ ]:
# 递归、记忆化搜索

import numpy as np
import sys

n = 30
memo = np.zeros(n) - 1
memo[0] = 1
memo[1] = 1

def rec_integerBreak_mem(n):
    maxProduct = -sys.maxsize - 1

    if n == 2 or n == 1:
        return 1
    
    if memo[n - 1] != -1:
        return memo[n - 1]
    
    for i in range(1, n):
        maxProduct = max(maxProduct, max(i * (n - i), i * rec_integerBreak_mem(n - i)))
    
    memo[n - 1] = maxProduct

    return maxProduct

print(rec_integerBreak_mem(n))

In [ ]:
# 动态规划
import numpy as np

def dp_integerBreak(n):
    memo = np.zeros(n + 1) - 1
    memo[1] = 1

    for i in range(2, n + 1):
        for j in range(1, i):
            memo[i] = max(memo[i], max(j * (i - j), j * memo[i - j]))
    
    return memo[-1]

print(dp_integerBreak(30))

## Perfect Squares

**Description：**

Given a positive integer n, find the **least number** of perfect square numbers (for example, 1, 4, 9, 16, ...) which sum to n.

**Example 1:**
```
Input: n = 12
Output: 3 
Explanation: 12 = 4 + 4 + 4.
```

**Example 2:**
```
Input: n = 13
Output: 2
Explanation: 13 = 4 + 9.
```

**题目分析：**

这是一道动态规划题目，要解决动态规划题目需要掌握三个信息：

- 最优子结构（最难的是发现最优子结构）
- 边界条件
- 根据最优子结构写出状态转移方程

**重叠子问题：**

我们用数组dp\[i\]表示第i个数字的完美平方数。我们来寻找重叠子问题：

我们要找到13的完美平方数，就把13拆成两个数的和，再分别找到这两个数的完美平方数，这就是这道题的重叠子问题。但是有一点我们不能忽略，如果这个13刚好是一个数的平方，那么它的完美平方数就是1。

**最优子结构分析：**

13的完美平方数 $= Min( dp[i]+dp[13-i] , dp[13])$ ; i = 1 到 13/2。

**状态转移方程：**

根据上面分析出来的最优子结构，我们可以写出来这道题的状态转移方程：
$$
dp[i] = min(dp[j] + dp[i - j], dp[i])
$$
i 的完全平方数是从和为 i 的两个完全平方数 dp\[j\] 和 dp\[i-j\] 之和 与 dp\[i\] 中取最小。

In [ ]:
# 递归

import math
import sys

def rec_numSquares(n):
    if n == 0:
        return 0
    
    count = sys.maxsize
    
    i = 1
    while i * i <= n:
        count = min(count, rec_numSquares(n - i * i) + 1)
        i += 1
    
    return count

print(rec_numSquares(12))

In [ ]:
# 递归，带“记忆”

import numpy as np
import sys

map = {}
def rec_numSquares_mem(n):
    if n in map:
        return map[n]
    
    if 0 == n:
        return 0
    
    count = sys.maxsize
    i = 1
    while i * i <= n:
        count = min(count, rec_numSquares_mem(n - i * i) + 1)
        i += 1
    map[n] = count
    return count

print(rec_numSquares_mem(12))

In [ ]:
# 动态规划

import numpy as np
import sys
import math

def dp_numSquares(n):
    dp = np.zeros(n + 1) + sys.maxsize
    dp[0] = 0

    for i in range(1, n + 1):
        j = 1
        while j * j <= i:
            dp[i] = min(dp[i], dp[i - j * j] + 1)
            j += 1
    
    return dp[n]

print(dp_numSquares(12))

## Decode Ways

**Description：**

A message containing letters from A-Z is being encoded to numbers using the following mapping:
```
'A' -> 1
'B' -> 2
...
'Z' -> 26
```
Given a **non-empty** string containing only digits, determine the total number of ways to decode it.

**Example 1:**
```
Input: "12"
Output: 2
Explanation: It could be decoded as "AB" (1 2) or "L" (12).
```

**Example 2:**
```
Input: "226"
Output: 3
Explanation: It could be decoded as "BZ" (2 26), "VF" (22 6), or "BBF" (2 2 6).
```

**解题思路：**

**（1）递归解法**

题目分析：

如果要用递归解法，就需要先找出递归公式：

numDecodings\[102213\] = numDecodings\[02213\] + numDecodings\[2213\] 

字符串“102213”解码的方式有两种：第一种，先解码子串“1”，再解码子串“02213”。但是以0开始的子串是无法解析的。第二种，先解码子串“10”，再解码子串“2213”。

解码子串“2213”也有两种方式：第一种，先解码子串“2”，再解码子串“213”。第二种，先解码子串“22”，再解码子串“13”。

以此类推。。。

规律显而易见，类似斐波那契数列。不同的是字符串从后往前推出递推公式。

**递归公式：**
```
numDecodings[s]= numDecodings[s.substring(1)] + numDecodings[s.substring(2)]
```

**递归结束条件：**

1. 当前指针所指字符为0

　　此时此字符无法解码，所以递归公式中的前者就只能为0，后者也为0。

　　例如【023】，substring（1）——【0】|【23】，截掉的【0】不能解析，所以此组合无效。

　　　　　　　　   substring（2）——【02】|【3】，截掉的【02】不能解析，所以此组合无效。

　　所以0数字出现后，解码数的增量为0。

2. 当前字符的值是有效的（大于0），但是当前字符与右边字符组合的数字无效（大于26）。

　　相当于递归公式中的后者为0。

　　例如【3212】，substring（1）——【3】|【212】，截掉的【3】能解析，所以其值为【212】的解码数。

　　　　　　　　     substring（2）——【32】|【12】，截掉的【32】不能解析，所以此组合无效。

In [ ]:
# 递归

def rec_numDecodings(s):
    if 0 == len(s):
        return 1
    if s[0] == '0':
        return 0
    if len(s) == 1:
        return 1
    res = rec_numDecodings(s[1:])
    if s[0] == '1' or s[0] == '2' and ord(s[1]) - ord('0') <= 6:
        res += rec_numDecodings(s[2:])
    return res

print(rec_numDecodings('102213'))

In [ ]:
# 记忆化递归

memo = {}
memo[''] = 1

def rec_numDecodings_mem(s):
    if s in memo:
        return memo[s]
    if len(s) == 0:
        return 1
    if s[0] == '0':
        return 0
    if len(s) == 1:
        return 1
    res = rec_numDecodings_mem(s[1:])
    if s[0] == '1' or s[0] == '2' and ord(s[1]) - ord('0') <= 6:
        res += rec_numDecodings_mem(s[2:])
    
    memo[s] = res
    return res

print(rec_numDecodings_mem('102213'))

## 动态规划解法

1. 从递归的解法中，我们发现了重叠子问题。根据重叠子问题，我们得到最优子结构。

假设数组dp\[i\]表示从头到字符串的第i位，一共有多少种解码方法的话。那么如果字符串的第i-1位和第i位能组成一个10到26的数字，说明我们是在第i-2位的解码方法上继续解码，此时dp\[i\] = dp\[i-2\]。如果字符串的第i-1位和第i位不能组成有效二位数字，而且第i位不是0的话，说明我们是在第i-1位的解码方法上继续解码，此时dp\[i\] = dp\[i - 1\]。所以，如果两个条件都符合，则dp\[i\] = dp\[i - 1\] + dp\[i - 2\]。

 2. 根据最优子结构，写出状态转移方程。

dp\[i\]的取值有以下四种情况：

1. 如果s\[i\]和s\[i - 1\]s\[i\]都是无效的话，dp\[i\] = 0;

2. 如果s\[i\]和s\[i - 1\]s\[i\]都是有效的话，dp\[i\] = dp\[i - 1\] + dp\[i - 2\];

3. 如果s\[i\]是有效的，s\[i - 1\]s\[i\]是无效的话，dp\[i\] = dp\[i - 1\];

4. 如果s\[i - 1\]s\[i\]是有效的，s\[i\]是无效的话，dp\[i\] = dp\[i - 2\];
- 如果 s\[i\] != '0'，则 s\[i\]是有效的。
- 如果‘10’<= s\[i - 1\]s\[i\] <='26'，则s\[i-1\]s\[i\]是有效的。

```
s = "102213"
```

|i|s\[i\]|s\[i-1\]s\[i\]|dp\[i\]|
|:---:|:---:|:---:|:---:|
|0|"1"|N/A|1|
|1|"0"|"10"|1|
|2|"2"|"02"|1|
|3|"2"|"22"|2|
|4|"1"|"21"|3|
|5|"3"|"13"|5|

In [ ]:
# 动态规划

import numpy as np

def dp_numDecodings(s):
    if len(s) == 0:
        return len(s)
    dp = np.zeros(len(s) + 1)
    dp[0] = 1
    # 如果第一位是0，则无法解码
    if s[0] == '0':
        dp[1] = 0
    else:
        dp[1] = 1
    for i in range(2, len(s) + 1):
        # 如果字符串的第i-1位和第i位能组成一个10到26的数字，说明我们可以在第i-2位的解码方法上继续解码
        if int(s[i-2 : i]) <= 26 and int(s[i-2 : i]) >= 10:
            dp[i] += dp[i - 2]
        # 如果字符串的第i-1位和第i位不能组成有效二位数字，在第i-1位的解码方法上继续解码
        if int(s[i-1 : i]) != 0:
            dp[i] += dp[i - 1]
    
    return dp[len(s)]

print(dp_numDecodings('102213'))

## Unique Paths

**Description：**

A robot is located at the top-left corner of a $m x n$ grid (marked 'Start' in the diagram below).

The robot can only move either down or right at any point in time. The robot is trying to reach the bottom-right corner of the grid (marked 'Finish' in the diagram below).

How many possible unique paths are there?

**Example 1:**
```
Input: m = 3, n = 2
Output: 3
Explanation:
From the top-left corner, there are a total of 3 ways to reach the bottom-right corner:
1. Right -> Right -> Down
2. Right -> Down -> Right
3. Down -> Right -> Right
```

**Example 2:**
```
Input: m = 7, n = 3
Output: 28
```

**解题思路：**

**（1）动态规划解法**

|1(Start)|1|1|1|1|1|1|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|1|2|3|4|5|6|7|
|1|3|6|10|15|21|28(Finish)|

**动态规划三要素：**

1. **最优子结构：**我们发现除去第一行第一列，其他二维数组中的数据都是由它上面格子和左边格子数据之和。

2. **边界条件：**第一行、第一列都为1。

3. **状态转移方程：** $dp[i][j] = dp[i-1][j] + dp[i][j-1]$

In [ ]:
# 递归

def dp_uniquePaths(m, n):
    if not m or not n:
        return 0
    dp = [[1 for i in range(m)] for i in range(n)]
    for i in range(1, n):
        for j in range(1, m):
            dp[i][j] = dp[i-1][j] + dp[i][j-1]
    return dp[-1][-1]

print(dp_uniquePaths(7, 3))

## Unique Paths II

**Description：**

A robot is located at the top-left corner of a $m x n$ grid (marked 'Start' in the diagram below).

The robot can only move either down or right at any point in time. The robot is trying to reach the bottom-right corner of the grid (marked 'Finish' in the diagram below).

Now consider if some obstacles are added to the grids. How many unique paths would there be?

An obstacle and empty space is marked as 1 and 0 respectively in the grid.

**Note:** m and n will be at most 100.

**Example 1:(())
```
Input:
[
  [0,0,0],
  [0,1,0],
  [0,0,0]
]
Output: 2
Explanation:
There is one obstacle in the middle of the 3x3 grid above.
There are two ways to reach the bottom-right corner:
1. Right -> Right -> Down -> Down
2. Down -> Down -> Right -> Right
```

**解题思路：**

**（1）动态规划**

|1(Start)|1|1|1|1|1|1|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|1|2|**0**|1|2|3|4|
|1|3|3|4|6|9|13(Finish)|

**动态规划三要素：**

1. **最优子结构：**我们发现除去第一行第一列，其他dp二维数组中的数据都是由它上面格子和左边格子数据之和。如果有障碍物，那么dp表格对应的位置为0。

2. **边界条件：**
- 如果obstacleGrid表格中第一行、第一列都没有障碍物，那么dp表格中第一行、第一列都为1。
- 如果start位置（即obstacleGrid\[0\]\[0\]==1）有障碍物，那么不管obstacleGrid表格后边是什么，路径和都为0。
- 如果obstacleGrid表格中第一行第i列有障碍物，那么dp表格中第一行第i列后边都为0。
- 如果obstacleGrid表格中第一列第i行有障碍物，那么dp表格中第一列第i行后边都为0。
3. **状态转移方程：** $dp[i][j] = dp[i-1][j] + dp[i][j-1]$

In [ ]:
# 动态规划

def dp_uniquePathsWithObstacles(obstacleGrid):
    if not obstacleGrid:
        return 0
    row = len(obstacleGrid)
    col = len(obstacleGrid[0])
    if obstacleGrid[0][0] == 1:
        return 0
    dp = [[1 for i in range(col)] for i in range(row)]
    for i in range(col):
        if obstacleGrid[0][i] == 1:
            for j in range(i, col):
                dp[0][j] = 0
            break
    for i in range(row):
        if obstacleGrid[i][0] == 1:
            for j in range(i, row):
                dp[j][0] = 0
            break
    for i in range(1, row):
        for j in range(1, col):
            if obstacleGrid[i][j] == 1:
                dp[i][j] = 0
            else:
                dp[i][j] = dp[i-1][j] + dp[i][j-1]
    return dp[-1][-1]

obstacleGrid = [[0,0],[1,1],[0,0]]
print(dp_uniquePathsWithObstacles(obstacleGrid))

obstacleGrid = [[0, 0, 0], [0, 1, 0], [0, 0, 0]]
print(dp_uniquePathsWithObstacles(obstacleGrid))

In [ ]:
# 更简洁的动态规划代码

def dp_uniquePathsWithObstacles_opt(obstacleGrid):
    row = len(obstacleGrid)
    col = len(obstacleGrid)
    dp = [[1] * col for _ in range(row)]

    for i in range(0, row):
        for j in range(0, col):
            if obstacleGrid[i][j]:
                dp[i][j] = 0
            elif i == 0:
                dp[i][j] = dp[i][j-1]
            elif j == 0:
                dp[i][j] = dp[i-1][j]
            else:
                dp[i][j] = dp[i-1][j] + dp[i][j-1]
    return dp[-1][-1]

obstacleGrid = [[0,0],[1,1],[0,0]]
print(dp_uniquePathsWithObstacles(obstacleGrid))

obstacleGrid = [[0, 0, 0], [0, 1, 0], [0, 0, 0]]
print(dp_uniquePathsWithObstacles(obstacleGrid))

## House Robber

**Description:**

You are a professional robber planning to rob houses along a street. Each house has a certain amount of money stashed, the only constraint stopping you from robbing each of them is that adjacent houses have security system connected and **it will automatically contact the police if two adjacent houses were broken into on the same night.**

Given a list of non-negative integers representing the amount of money of each house, determine the maximum amount of money you can rob tonight **without alerting the police.**

**Example 1:**
```
Input: [1,2,3,1]
Output: 4
Explanation: Rob house 1 (money = 1) and then rob house 3 (money = 3).
             Total amount you can rob = 1 + 3 = 4.
```

**Example 2:**
```
Input: [2,7,9,3,1]
Output: 12
Explanation: Rob house 1 (money = 2), rob house 3 (money = 9) and rob house 5 (money = 1).
             Total amount you can rob = 2 + 9 + 1 = 12.
```

等同于“数组不相邻元素最大和”问题。

## House Robber II

**Description：**

You are a professional robber planning to rob houses along a street. Each house has a certain amount of money stashed. All houses at this place are **arranged in a circle**. That means the first house is the neighbor of the last one. Meanwhile, adjacent houses have security system connected and **it will automatically contact the police if two adjacent houses were broken into on the same night.**

Given a list of non-negative integers representing the amount of money of each house, determine the maximum amount of money you can rob tonight **without alerting the police.**

**Example 1:**
```
Input: [2,3,2]
Output: 3
Explanation: You cannot rob house 1 (money = 2) and then rob house 3 (money = 2),
             because they are adjacent houses.
```

**Example 2:**
```
Input: [1,2,3,1]
Output: 4
Explanation: Rob house 1 (money = 1) and then rob house 3 (money = 3).
             Total amount you can rob = 1 + 3 = 4. 
```

**解题思路：**

$[6, 6, 0, 3, 1, 9]$

**分析：**

由于最后一个房子和第一个房子相邻，那么就会出现两种情况：

1）抢劫第一个房子的时候，不能抢劫最后一个房子。这种情况下，就是从输入序列中去除最后一个房子，也就是求输入序列的子序列中抢劫钱数最多，即求序列$[6, 6, 0, 3, 1]$。

2）抢劫最后一个房子的时候，不能抢劫第一个房子。这种情况下，就是从输入序列去除第一个房子，也就是求输入序列的子序列中抢劫钱数最多，即求序列$[6, 0, 3, 1, 9]$。

因此：

通过对上面的两种情况的分析，每一种情况都是上一道题（House Robber）的输入，我们可以把这道题的结果封装成一个函数，对上面两种情况分别进行输入计算即可。

In [ ]:
# 动态规划

def dp_rob(arr):
    if len(arr) == 0:
        return 0
    if len(arr) == 1:
        return arr[0]
    if len(arr) == 2:
        return max(arr)

    def getRob(start, stop, arr):
        nums = arr[start : stop + 1]
        dp = [0] * len(nums)
        dp[0], dp[1] = nums[0], max(nums[0], nums[1])
        for i in range(2, len(nums)):
            dp[i] = max(dp[i - 1], dp[i - 2] + nums[i])
        return dp[-1]
    
    return max(getRob(0, len(arr) - 2, arr), getRob(1, len(arr) - 1, arr))

arr = [6, 6, 0, 3, 1, 9]
print(dp_rob(arr))

## House Robber |||

**Description：**

The thief has found himself a new place for his thievery again. There is only one entrance to this area, called the "root." Besides the root, each house has one and only one parent house. After a tour, the smart thief realized that "all houses in this place forms a binary tree". It will automatically contact the police if two directly-linked houses were broken into on the same night.

Determine the maximum amount of money the thief can rob tonight without alerting the police.

**Example 1:**
```
Input: [3,2,3,null,3,null,1]

    [3]
    / \
   2   3
    \   \ 
    [3] [1]

Output: 7 
Explanation: Maximum amount of money the thief can rob = 3 + 3 + 1 = 7.
```
**Example 2:**
```
Input: [3,4,5,1,3,null,1]

     3
    / \
  [4] [5]
  / \   \ 
 1   3   1

Output: 9
Explanation: Maximum amount of money the thief can rob = 4 + 5 = 9.
```

**解题思路：**

**（1）递归法：**

关于树的问题，首先想到的是用递归法求解。针对每一个树节点我们给定一个长度为2的result列表。result\[0\]表示不抢劫当前节点能取到的最大值；result\[1\]表示抢劫当前节点能取到的最大值。

In [ ]:
# 定义二叉树节点
class TreeNode(object):
    """节点类"""
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

# 定义二叉树
class Tree(object):
    # 建立二叉树是以层序遍历方式输入，节点不存在时以'None'表示
    def creatTree(self, nodeList):
        if nodeList[0] == None:
            return None
        head = TreeNode(nodeList[0])
        Nodes = [head]
        j = 1
        for node in Nodes:
            if node != None:
                node.left = (TreeNode(nodeList[j]) if nodeList[j] != None else None)
                Nodes.append(node.left)
                j += 1
                if j == len(nodeList):
                    return head
                node.right = (TreeNode(nodeList[j]) if nodeList[j] != None else None)
                j += 1
                Nodes.append(node.right)
                if j == len(nodeList):
                    return head
def rob(node):
    if node == None:
        return [0, 0]
    # result[0] is max value if not rob current node
    # result[1] is max value if rob current node
    result = [0] * 2
    left_val = rob(node.left)
    right_val = rob(node.right)
    result[0] = max(left_val[0], left_val[1]) + max(right_val[0], right_val[1])
    result[1] = left_val[0] + right_val[0] + node.val
    return result

elements = [3, 2, 3, None, 3, None, 1]
tree = Tree()
head = tree.creatTree(elements)
print(rob(head))

## 汉诺塔

In [ ]:
# 递归

nstep = 0
def hanoi(n, start, end):
    # n是总的块数
    # start是开始挪动的柱子
    # end是要挪去的柱子
    # 假设start和end在1～3之间，且互不相等
    # nstep是总移动次数
    global nstep
    if n == 1:
        pm(start, end)
        nstep += 1
        return nstep
    else:
        other = 6 - (start + end)
        hanoi(n - 1, start, other)
        pm(start, end)
        nstep +=1
        hanoi(n - 1, other, end)
    return nstep

def pm(start, end):
    print(start, ' -> ', end)

print(hanoi(3, 1, 3))

## Longest Increasing Subsequence (LIS)

For a sequence $a_1,\ a_2,\ \dots,\ a_n$, find the length of the longest increasing subsequence $a_{i1},\ a_{i2},\ \dots,\ a_{ik}$

**Constraints:** $i_1 < i_2 < \dots < i_k;\ a_{i1} < a_{i2} < \dots < a_{ik}$

$$
LIS([3\quad 1\quad 8\quad 2\quad 5])\rightarrow[1\quad 2\quad 5]\rightarrow 3\\
LIS(5\quad 2\quad 8\quad 6\quad 3\quad 6\quad 9\quad 5)\rightarrow[2\quad 3\quad 6\quad 9]\rightarrow 4
$$

We will focus on the **length** of the LIS

**Find an appropriate subproblem**

- All increasing subsequences are subsets of original sequence.
- All increasing subsequences have a *start* and *end*.

Let's focus on the *end* index of an increasing subsequence.

Subproblem: LIS\[$k$\] = LIS ending at index $k$

e.g. \[3 1 8 2 5\]

LIS\[3\] = LIS\[3 1 8 2\] = \[1 2\] = 2

**Find relationships among subproblems**

What subproblems are needed to solve **LIS\[4\]**?

LIS\[0\] = 1, LIS\[1\] = 1, LIS\[3\] = 2

LIS\[4\] = 1 + max{LIS\[0\], LIS\[1\], LIS\[3]}

A = \[5 2 8 6 3 6 9 5\]
$$
\begin{align}
LIS[5] &= 1 + max\left\{LIS[k]\quad |\quad k < 5,\ A[k] < A[5]\right\} \\
       &= 1 + max\left\{LIS[0],\ LIS[1],\ LIS[4]\right\}
\end{align}
$$

**递归式:**
$LIS[n] = 1 + max\left\{LIS[k]\quad |\quad k < n,\ A[k] < A[n]\right\}$

In [ ]:
def lis(A):
    L = [1] * len(A)
    for i in range(1, len(L)):
        subproblems = [L[k] for k in range(i) if A[k] < A[i]]
        L[i] = 1 + max(subproblems, default=0)
    return max(L, default=0)

How do we actually get the sequence?

Keep track of previous indices!

$prev[i] = j$

Previous index used to solve LIS\[i\] is index $j$

$prev[0] = -1,\quad prev[1] = -1,\quad prev[2] = 0,\quad prev[3] = 1,\quad prev[4] = 3$